In [1]:
from glob import glob
import folium

lon, lat = -86.276, 30.935 
zoom_start = 5

mapa = folium.Map(location=[lat, lon], tiles="Cartodb Positron",
                  zoom_start=zoom_start)

In [2]:
import json

with open('data/stations.geojson') as f:
    stations = json.load(f)

with open('data/hfradar.geojson') as f:
    hfradar = json.load(f)

In [3]:
icon_size = (14, 14)

for feature in stations['features']:
    lon, lat = feature['geometry']['coordinates']
    icon_url = feature['properties']['icon']
    popup = feature['properties']['popupContent']
    
    icon = folium.features.CustomIcon(icon_url,
                                      icon_size=(14, 14))
    
    marker = folium.map.Marker([lat, lon], icon=icon,
                               popup=folium.map.Popup(popup))
    mapa.add_children(marker)

In [4]:
for feature in hfradar['features']:
    if feature['geometry']['type'] == 'Point':
        lon, lat = feature['geometry']['coordinates']
        icon_url = feature['properties']['icon']
        popup = feature['properties']['popupContent']
        
        icon = folium.features.CustomIcon(icon_url,
                                          icon_size=(14, 14))
        marker = folium.map.Marker([lat, lon], icon=icon,
                                   popup=folium.map.Popup(popup))
        mapa.add_children(marker)
    elif feature['geometry']['type'] == 'Polygon':
        # FIXME: We need a clean way to use:
        # https://github.com/mapbox/simplestyle-spec/
        # Issue: https://github.com/python-visualization/folium/issues/120
        kw = dict(fill=feature['properties']['fill'],
          fill_opacity=feature['properties']['fill_opacity'],
          stroke=feature['properties']['stroke'],
          stroke_opacity=feature['properties']['stroke_opacity'],
          stroke_width=feature['properties']['stroke_width'])
        gjson = folium.features.GeoJson(feature)
        
        mapa.add_children(gjson)

In [5]:
mapa